# preprocess fibsemdata

In [1]:
import dask.array as da
from dask import delayed
import napari 
import numpy as np
from fst.io import read, access_n5
import matplotlib.pyplot as plt
from distributed import Client
from fish.fish.util.distributed import get_jobqueue_cluster
from skimage.exposure import rescale_intensity as rescale
import numcodecs
%gui qt

/groups/scicompsoft/home/bennettd/miniconda3/lib/python3.7/site-packages/napari/__init__.py:27: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.9.7. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


In [2]:
def shifter(data, shifts, block_id=None, order=1, cval=0.0):  
    from scipy.ndimage.interpolation import shift
    shifts_ = (0, *shifts[block_id[0]])   
    return shift(data, shifts_, order=order, cval=cval).astype('float32')

def detrend_spline(data, **kwargs):
    from numpy import stack, arange, rollaxis
    from scipy.interpolate import LSQUnivariateSpline
    x = arange(data.shape[0])
    result = stack([LSQUnivariateSpline(x, y, **kwargs)(x) for y in rollaxis(data, 1)]).T
    return data - result

In [3]:
from pathlib import Path
import zarr
exp_path = '/groups/cosem/cosem/data/LoadID365_ROI4_8x8x8nm/LoadID365_ROI4_8x8x8nm.n5/'
output_path = '/volumes/proc/'
input_path = '/volumes/raw/ch0'
exp_name = Path(exp_path).parts[-1].split('.')[0]
dlog = zarr.open(f'/nrs/cosem/davis/{exp_name}_procdata.zr')

zr = read(exp_path + input_path)
data = da.from_array(zr, chunks=(1,-1,-1))
binning = {0:2, 1:1, 2:1}
output_pixel_size = [8,8,8]
pixel_unit = 'nm'

attrs = dict()
attrs['name'] = exp_path.split('/')[-2].split('.')[0]
attrs['pixelResolution'] = dict(dimensions=output_pixel_size,
                                unit=pixel_unit)

In [4]:
output_dtype = 'uint16'
output_chunks = data.ndim * (256,)
raw_shifts = dlog['shifts/raw'][:]
raw_shifts_det = detrend_spline(raw_shifts, t = np.arange(1, raw_shifts.shape[0], 1000))
accumulated_shifts = np.cumsum(raw_shifts,0)
accumulated_shifts_det = np.cumsum(raw_shifts_det, 0)

shifts_dsk = da.from_array(accumulated_shifts_det, chunks=(-1,-1))

In [5]:
mn, mx = data.min(), data.max()

flipped = data.map_blocks(rescale, 
                         in_range=(mn, mx), 
                         out_range=(mx, mn), 
                         dtype=data.dtype)

shifted = flipped.map_blocks(shifter, 
                             shifts=shifts_dsk, 
                             cval=mx,
                             order=3,
                             dtype='float32')

binned = da.coarsen(np.mean, 
                    shifted.rechunk((2,-1,-1)), 
                    binning)

rescaled = binned.map_blocks(rescale, 
                             in_range=(binned.min(), binned.max()), 
                             out_range=output_dtype, 
                             dtype=binned.dtype).astype(output_dtype)

rechunked = rescaled.rechunk(output_chunks)


In [6]:
# save to disk
from numcodecs import GZip
compressor=GZip(level=-1)
dest = access_n5(dir_path=exp_path, 
                 container_path=output_path,
                 shape=rechunked.shape,
                 mode='w',
                 chunks=output_chunks,
                 compressor=compressor,
                 dtype=rechunked.dtype)

dest.attrs.update(**attrs)

In [7]:
client = Client(get_jobqueue_cluster())
client

/groups/scicompsoft/home/bennettd/miniconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://10.36.111.12:46873 Dashboard: http://10.36.111.12:42389/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [8]:
client.cluster.start_workers(100)
#sample = flipped[0].compute()
rechunked.to_zarr(dest)
client.cluster.stop_all_jobs()

In [1]:
# put in the chmodr call